In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import tensorflow as tf
from sklearn.model_selection import train_test_split
import scipy.misc
from tensorflow.keras.models import load_model

image = tf.keras.preprocessing.image

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
output_layer_classes = 196

In [4]:
base_path = '/home/maxim/notebooks/ObjectRecognition/datasets'

def get_path(path):
    return os.path.join(base_path, path)

In [5]:
mat = scipy.io.loadmat(get_path('devkit/cars_meta.mat'))
label_names = np.array([row[0] for row in mat['class_names'].reshape(-1)])

In [6]:
mat = scipy.io.loadmat(get_path('devkit/cars_annos.mat'))
file_labels = {row[0][0]: row[5].squeeze() - 1 for row in mat['annotations'][0]}

filenames_np = np.array([get_path(row[0][0]) for row in mat['annotations'][0]])

labels_np = np.array([row[5].squeeze() - 1 for row in mat['annotations'][0]])
labels_one_hot = np.zeros((labels_np.size, output_layer_classes))
labels_one_hot[np.arange(labels_np.size), labels_np] = 1

In [7]:
filenames_train_np, filenames_test_np, labels_train_np, labels_test_np = \
    train_test_split(filenames_np, labels_one_hot, test_size=0.05, random_state=42)

In [8]:
filenames_train = tf.constant(filenames_train_np)
labels_train = tf.constant(labels_train_np)
filenames_test = tf.constant(filenames_test_np)
labels_test = tf.constant(labels_test_np)

In [9]:
dataset_train = tf.data.Dataset.from_tensor_slices((filenames_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((filenames_test, labels_test))

image_count = int(filenames_train.shape[0])
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3
BATCH_SIZE = 64
STEPS_PER_EPOCH = np.ceil(2 * image_count / BATCH_SIZE)


def convert(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
     # img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    return img, label

def augment(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
     # img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.cast(img, tf.float32)
    img = (img / 127.5) - 1
    img = tf.image.resize(img, [2 * IMG_WIDTH, 2 * IMG_HEIGHT])
    
    img = tf.image.random_crop(img, size=[IMG_WIDTH, IMG_HEIGHT, CHANNELS]) # Random crop back to 28x28
    img = tf.image.random_brightness(img, max_delta=0.5) # Random brightness
    img = tf.image.random_flip_left_right(img)

    return img,label

database_train1 = dataset_train.map(convert)
database_train2 = dataset_train.map(augment)
database_train = database_train1.concatenate(database_train2)

database_test = dataset_test.map(convert)

In [10]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(label_names[label_batch[n]])
        plt.axis('off')


In [11]:
train_ds = prepare_for_training(database_train, cache='cache')
test_ds = prepare_for_training(database_test)

In [12]:
model = load_model('cars_full_aug12_double_data.h5')

In [13]:

img_path = get_path('AUDI-A5-2.jpg')
img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 127.5 - 1
print('Input image shape:', x.shape)
#my_image = scipy.misc.imread(img_path)
#imshow(my_image)
#print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
pred = np.array(model.predict(x)).reshape(-1)
label_names[np.argmax(pred)]

Input image shape: (1, 224, 224, 3)


'Audi A5 Coupe 2012'

In [14]:
model2= tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

In [15]:
#model2.save('model2.h5')

In [16]:
def vectorize(filename):
    img_path = get_path(filename)
    img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 127.5 - 1
    #print(x)
    return np.array(model2.predict(x)).reshape(-1)

In [29]:
import scipy.linalg as sla
from scipy import spatial

In [18]:
hummer1 = vectorize('hummer1.jpg')
hummer2 = vectorize('hummer2.jpg')
audi1 = vectorize('AUDI-A5.jpg')
audi2 = vectorize('AUDI-A5-2.jpg')
camry1 = vectorize('camry.jpg')
camry2 = vectorize('camry2.jpg')
mb = vectorize('mercedes-e-classe.jpg')
santafe1 = vectorize('hyundai-santa-fe.jpg')
santafe2 = vectorize('santafe2.jpg')

In [30]:
sla.norm(hummer2 - audi2), spatial.distance.cosine(hummer2, audi2)

(41.39241409301758, 0.9359222948551178)

In [31]:
sla.norm(hummer1 - audi1), spatial.distance.cosine(hummer1, audi1)

(32.20895004272461, 0.7749945968389511)

In [32]:
sla.norm(audi2 - audi1), spatial.distance.cosine(audi2, audi1)

(27.448389053344727, 0.4050180912017822)

In [33]:
sla.norm(hummer2 - hummer1), spatial.distance.cosine(hummer2, hummer1)

(19.293447494506836, 0.2614699602127075)

In [34]:
sla.norm(hummer1 - camry1), spatial.distance.cosine(hummer1, camry1)

(31.337045669555664, 0.7573308944702148)

In [37]:
sla.norm(camry1 - camry2), spatial.distance.cosine(camry1, camry2)

(19.24966049194336, 0.2142622470855713)

In [38]:
sla.norm(santafe1 - mb), spatial.distance.cosine(santafe1, mb)

(31.648765563964844, 0.4753033518791199)

In [39]:
sla.norm(santafe2 - mb), spatial.distance.cosine(santafe2, mb)

(31.84998893737793, 0.5186568796634674)

In [40]:
sla.norm(santafe2 - santafe1), spatial.distance.cosine(santafe2, santafe1)

(17.136764526367188, 0.1515343189239502)

In [97]:
hummer1.shape

(1280,)